# Prompt Injection Attacker — single notebook
**Author:** Nebiyu Zewge
**Date:** 2025-11-04

**WARNING:** This notebook will generate malicious prompts and may produce harmful outputs. Use only for research in a safe environment. Do not run on public endpoints or use generated prompts for malicious activity.


In [2]:
# Requirements cell - installs packages if missing, then imports
import sys
import subprocess
import importlib.util


def install(pkg: str):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", pkg])


# (pip_name, module_name) to avoid deprecation and handle mismatched names (e.g., scikit-learn -> sklearn)
PACKAGES = [
    ("transformers", "transformers"),
    ("torch", "torch"),
    ("faker", "faker"),
    ("pandas", "pandas"),
    ("tqdm", "tqdm"),
    ("matplotlib", "matplotlib"),
    ("numpy", "numpy"),
    ("scikit-learn", "sklearn"),  # module is 'sklearn'
    ("ipywidgets", "ipywidgets"),  # for nice tqdm notebook progress bars
]

for pip_pkg, mod_name in PACKAGES:
    if importlib.util.find_spec(mod_name) is None:
        print(f"Installing {pip_pkg} ...")
        install(pip_pkg)

# Imports
import os
import time
import json
import base64
import re
from faker import Faker
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Optional: print versions for reproducibility
try:
    import transformers, sklearn, faker, tqdm as _tqdm
    print(f"Python: {sys.version.split()[0]}")
    print(f"torch: {torch.__version__} | CUDA available: {torch.cuda.is_available()}")
    print(f"transformers: {transformers.__version__}")
    print(f"pandas: {pd.__version__}")
    print(f"numpy: {np.__version__}")
    print(f"matplotlib: {matplotlib.__version__}")
    print(f"faker: {faker.__version__}")
    print(f"tqdm: {_tqdm.__version__}")
    print(f"scikit-learn: {sklearn.__version__}")
except Exception as _e:
    # Avoid hard failure if version attributes are missing
    pass


Installing ipywidgets ...
Python: 3.13.5
torch: 2.7.1+cpu | CUDA available: False
transformers: 4.57.1
pandas: 2.3.2
numpy: 2.2.6
matplotlib: 3.10.6
Python: 3.13.5
torch: 2.7.1+cpu | CUDA available: False
transformers: 4.57.1
pandas: 2.3.2
numpy: 2.2.6
matplotlib: 3.10.6


In [4]:
# Configuration (Step-3)
# Central place to change models, generation params, paths, and GPU usage
MODEL_NAMES = ["gpt2", "distilgpt2", "gpt2-medium"]  # change as needed
USE_GPU = torch.cuda.is_available()
DEVICE = "cuda" if USE_GPU else "cpu"
DEVICE_ID = 0 if USE_GPU else -1  # pipeline device id if GPU; else -1 for CPU

GEN_PARAMS = {
    "max_length": 120,
    "temperature": 0.9,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 1,
}

RUN_ID = time.strftime("run_%Y%m%d_%H%M%S")
OUT_DIR = f"results/{RUN_ID}"
os.makedirs(OUT_DIR, exist_ok=True)

print(f"Device: {DEVICE} (cuda available: {torch.cuda.is_available()})")
print(f"Models: {MODEL_NAMES}")
print(f"Outputs: {OUT_DIR}")


Device: cpu (cuda available: False)
Models: ['gpt2', 'distilgpt2', 'gpt2-medium']
Outputs: results/run_20251104_094733
